In [200]:
# нью рок нью джаз
import numpy as np
import pandas as pd

In [201]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

imputer = SimpleImputer(strategy="most_frequent")

In [202]:
path_train = "train"

wag_prob = pd.read_parquet(path_train + "/wagons_probeg_ownersip.parquet").convert_dtypes()
target = pd.read_csv(path_train + '/target/y_train.csv').convert_dtypes()
to_predict = pd.read_csv(path_train + '/target/y_predict.csv').convert_dtypes()
wag_param = pd.read_parquet(path_train + "/wag_params.parquet").convert_dtypes()
# добавляем kri_izm
kri_izm = pd.read_parquet(path_train + '/kti_izm.parquet').convert_dtypes()

In [203]:
train_target = target[target["month"] == "2023-01-01"].drop("month", axis=1)
train = wag_prob[wag_prob["repdate"] < "2023-01-01"]

path_test = 'test'
test_target = pd.read_csv(path_test + '/target/y_test.csv').convert_dtypes().drop("month", axis=1)
test = wag_prob


In [204]:
def prepare_data(group_data, wag_param, target, columns_to_drop=None,
                 columns_to_drop_wag_param=None,
                 imputer_strategy='most_frequent'):
    # Group and aggregate the data
    if columns_to_drop_wag_param is None:
        columns_to_drop_wag_param = ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"]

    features = group_data.groupby("wagnum").agg(
        {
            "ost_prob": ["mean", "last", "max", 'var'],
            "manage_type": "last",
            "rod_id": "last",
            "reestr_state": "last",
        }
    )
    features.columns = ["_".join(col).strip() for col in features.columns.values]
    features = features.reset_index()

    # Drop unnecessary columns from wag_param
    m_data = wag_param.drop(["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1)

    # Merge the aggregated features with the wag_param
    merged = features.merge(m_data, on="wagnum", how="inner")

    # If a target dataframe is provided, merge it as well
    if target is not None:
        merged = target.merge(merged, on="wagnum", how="left")

    # Define and apply an imputer for missing values
    imputer = SimpleImputer(strategy=imputer_strategy)

    column_names = merged.columns
    merged = imputer.fit_transform(merged)
    merged = pd.DataFrame(merged, columns=column_names)

    if columns_to_drop is not None:
        merged = merged.drop(columns_to_drop, axis=1)

    # Drop the 'wagnum' column to prepare the feature matrix
    X_data = merged.drop(["wagnum"], axis=1)

    return X_data

In [205]:


# Define the columns to drop from wag_param
columns_to_drop = ['tippogl',
                   'tormoz',
                   'norma_km',
                   'kuzov',
                   'telega',
                   'cnsi_probeg_dr',
                   'manage_type_last',
                   'ownertype',
                   'cnsi_probeg_kr',
                   'reestr_state_last', 'zavod_build',
                   'cnsi_gruz_capacity',
                   'rod_id',
                   'gruz',
                   'rod_id_last']

# Prepare training data
X_train = prepare_data(train, wag_param, train_target, columns_to_drop)

# Prepare testing data
X_test = prepare_data(test, wag_param, test_target, columns_to_drop)


In [206]:
X_train.head()

,target_month,target_day,ost_prob_mean,ost_prob_last,ost_prob_max,ost_prob_var,cnsi_volumek,tara
0,0.0,0.0,40308.352941,153692.0,159999.0,4.462370e+09,88.0,245.0
1,0.0,0.0,36773.124183,157438.0,159999.0,1.860820e+09,88.0,245.0
2,0.0,0.0,30174.464052,159999.0,159999.0,3.164737e+09,88.0,245.0
3,0.0,0.0,34319.823529,28597.0,41827.0,1.793441e+07,88.0,245.0
4,0.0,0.0,85561.934641,150043.0,159999.0,5.815368e+09,88.0,245.0


In [207]:
X_test.head()

,target_month,target_day,ost_prob_mean,ost_prob_last,ost_prob_max,ost_prob_var,cnsi_volumek,tara
0,0.0,0.0,59127.777174,150961.0,159999.0,5.464044e+09,88.0,245.0
1,0.0,0.0,56391.706522,148432.0,159999.0,3.456643e+09,88.0,245.0
2,0.0,0.0,51519.869565,154991.0,159999.0,4.890354e+09,88.0,245.0
3,0.0,0.0,33082.565217,24859.0,41827.0,2.280311e+07,88.0,245.0
4,0.0,0.0,96083.864130,144208.0,159999.0,5.380395e+09,88.0,245.0


In [208]:

y_month_name = "target_month"
y_day_name = "target_day"
target_names = [y_month_name, y_day_name]

### Предобработка КТИ

In [209]:
kti_izm = pd.read_parquet('train/kti_izm.parquet').convert_dtypes()
kti_izm.head()

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,325,2022-08-01,112091,30.5,31.0,26.7,27.4,28.8,28.6,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
1,325,2022-08-03,112471,30.4,31.4,27.5,27.6,28.9,29.0,25.5,27.8,65.7,65.8,65.7,63.7,61.6,61.4,67.4,69.3
2,325,2022-08-05,113938,30.0,31.1,27.1,27.9,28.5,28.9,26.2,27.6,70.8,69.5,65.1,65.6,62.2,60.7,65.7,69.6
3,325,2022-09-15,121071,30.5,31.4,26.4,27.3,28.9,28.4,24.9,28.0,70.2,68.1,67.4,65.8,62.3,63.0,70.9,70.3
4,325,2022-09-09,117341,30.7,31.5,26.9,27.1,28.9,28.5,25.5,27.5,68.1,64.1,64.5,64.9,61.6,61.1,68.7,<NA>


In [210]:
# from matplotlib import pyplot as plt
# 
# selected_wagnum = kti_izm['wagnum'].unique()[0]
# selected_df = kti_izm[kti_izm['wagnum'] == selected_wagnum]
# 
# # Получение списка параметров, исключая 'wagnum' и 'operation_date'
# parameters = selected_df.columns.drop(['wagnum', 'operation_date_dttm'])
# 
# # Определение размера сетки графиков
# n_params = len(parameters)
# n_cols = 3  # Количество колонок графиков
# n_rows = (n_params + n_cols - 1) // n_cols  # Количество строк графиков
# 
# fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))  # Размер фигуры в дюймах
# 
# # Создание графиков в цикле
# for i, param in enumerate(parameters):
#     row = i // n_cols
#     col = i % n_cols
#     ax = axs[row, col] if n_rows > 1 else axs[col]
#     ax.plot(selected_df['operation_date_dttm'], selected_df[param], label=param)
#     ax.set_title(f'Изменение {param} со временем')
#     ax.set_xlabel('Дата операции')
#     ax.set_ylabel(param)
#     ax.grid(True)
#     ax.legend()
# 
# # Удаление пустых мест для графиков, если параметров меньше, чем мест в сетке
# for i in range(n_params, n_rows * n_cols):
#     row = i // n_cols
#     col = i % n_cols
#     fig.delaxes(axs[row, col] if n_rows > 1 else axs[col])
# 
# plt.tight_layout()
# plt.show()

In [211]:
# kti_izm_agg = kti_izm.drop('operation_date_dttm', axis=1).groupby("wagnum").agg(['max', 'min', 'mean', 'std', 'var', 'last'])
# kti_izm_agg.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in kti_izm_agg.columns.values]
# kti_izm_agg.head()

In [212]:

# добавление КТИ
# merged = pd.merge(merged, kti_izm_agg, on='wagnum', how='left')
# #merged.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in merged.columns.values]
# 
# merged['is_kti_found'] = merged['mileage_all_max'].notnull().astype(int)



In [213]:

y_month_name = "target_month"
y_day_name = "target_day"
target_names = [y_month_name, y_day_name]

In [214]:
print("Training for month and day prediction")

logreg_month = LogisticRegression()
logreg_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
logreg_day = LogisticRegression()
logreg_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

rf_month = RandomForestClassifier()
rf_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
rf_day = RandomForestClassifier()
rf_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])




Training for month and day prediction


RandomForestClassifier()

In [215]:
# Подсчитаем количество примеров каждого класса
class_counts = X_train[y_month_name].value_counts()
# Например, если у нас два класса, то вес для каждого класса может быть обратно пропорционален его частоте.
class_weights = class_counts.sum() / (len(class_counts) * class_counts)
cb_month = CatBoostClassifier(random_seed=42
                              #auto_class_weights='Balanced',
                              #eval_metric='F1', 

                              )
cb_month.fit(
    X_train.drop(target_names, axis=1),
    X_train[y_month_name],
    #eval_set=(X_test.drop(target_names, axis=1), X_test[y_month_name]),
    verbose=True, )

# Подсчитаем количество примеров каждого класса
class_counts = X_train[y_day_name].value_counts()
# Например, если у нас два класса, то вес для каждого класса может быть обратно пропорционален его частоте.
class_weights = class_counts.sum() / (len(class_counts) * class_counts)
cb_day = CatBoostClassifier(random_seed=42,
                            auto_class_weights='Balanced',
                            #eval_metric='F1',
                            iterations=1000)
cb_day.fit(
    X_train.drop(target_names, axis=1),
    X_train[y_day_name],
    eval_set=(X_test.drop(target_names, axis=1), X_test[y_day_name]),
    verbose=True,
    use_best_model=False
)

Learning rate set to 0.046422
0:	learn: 0.5664623	total: 5.09ms	remaining: 5.09s
1:	learn: 0.4923260	total: 11.3ms	remaining: 5.66s
2:	learn: 0.4307503	total: 16.9ms	remaining: 5.62s
3:	learn: 0.3889206	total: 21.2ms	remaining: 5.28s
4:	learn: 0.3420453	total: 26.7ms	remaining: 5.31s
5:	learn: 0.3014014	total: 31.6ms	remaining: 5.24s
6:	learn: 0.2639806	total: 37.5ms	remaining: 5.33s
7:	learn: 0.2376054	total: 43.6ms	remaining: 5.4s
8:	learn: 0.2143829	total: 49.4ms	remaining: 5.44s
9:	learn: 0.1998512	total: 55.5ms	remaining: 5.5s
10:	learn: 0.1802443	total: 62ms	remaining: 5.57s
11:	learn: 0.1680060	total: 66.9ms	remaining: 5.51s
12:	learn: 0.1590389	total: 71.7ms	remaining: 5.44s
13:	learn: 0.1493195	total: 77ms	remaining: 5.42s
14:	learn: 0.1425697	total: 83ms	remaining: 5.45s
15:	learn: 0.1340228	total: 89.6ms	remaining: 5.51s
16:	learn: 0.1286555	total: 95.4ms	remaining: 5.52s
17:	learn: 0.1252441	total: 104ms	remaining: 5.66s
18:	learn: 0.1206931	total: 112ms	remaining: 5.77s
19

In [216]:
y_pred_logreg_month = logreg_month.predict(X_test.drop(target_names, axis=1))
y_pred_rf_month = rf_month.predict(X_test.drop(target_names, axis=1))
y_pred_cb_month = cb_month.predict(X_test.drop(target_names, axis=1))

y_pred_logreg_day = logreg_day.predict(X_test.drop(target_names, axis=1))
y_pred_rf_day = rf_day.predict(X_test.drop(target_names, axis=1))
y_pred_cb_day = cb_day.predict(X_test.drop(target_names, axis=1))

print("Month score:")
f1_logreg_month = f1_score(X_test[y_month_name], y_pred_logreg_month)
print(f"Logistic Regression month F1 Score: {f1_logreg_month}")
f1_rf_month = f1_score(X_test[y_month_name], y_pred_rf_month)
print(f"Random Forest month F1 Score: {f1_rf_month}")
f1_cb_month = f1_score(X_test[y_month_name], y_pred_cb_month)
print(f"CatBoost month F1 Score: {f1_cb_month}")

print("Day score:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day)
print(f"CatBoost day F1 Score: {f1_cb_day}")

Month score:
Logistic Regression month F1 Score: 0.0
Random Forest month F1 Score: 0.5165609584214235
CatBoost month F1 Score: 0.515972222222222
Day score:
Logistic Regression day F1 Score: 0.0
Random Forest day F1 Score: 0.2825822168087698
CatBoost day F1 Score: 0.35111876075731496


In [217]:
def correct_days_with_month(day_array, month_array):
    corrected_day = day_array.copy()
    corrected_day[month_array == 0] = 0
    return corrected_day


y_pred_logreg_day_corrected = correct_days_with_month(y_pred_logreg_day, y_pred_logreg_month)
y_pred_rf_day_corrected = correct_days_with_month(y_pred_rf_day, y_pred_rf_month)
y_pred_cb_day_corrected = correct_days_with_month(y_pred_cb_day, y_pred_cb_month)

print("Day score after applying month info to day:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day_corrected)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day_corrected)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day_corrected)
print(f"CatBoost day F1 Score: {f1_cb_day}")


Day score after applying month info to day:
Logistic Regression day F1 Score: 0.0
Random Forest day F1 Score: 0.27552275522755226
CatBoost day F1 Score: 0.3601895734597157


In [218]:
X_test.drop(target_names, axis=1).columns

Index(['ost_prob_mean', 'ost_prob_last', 'ost_prob_max', 'ost_prob_var',
       'cnsi_volumek', 'tara'],
      dtype='object')

In [219]:
feature_importances = cb_month.get_feature_importance()
feature_names = X_test.drop(target_names, axis=1).columns
importance_dict = {feature_names[i]: importance for i, importance in enumerate(feature_importances)}
sorted_f_imp = dict(sorted(importance_dict.items(), key=lambda item: item[1], reverse=True))

In [220]:
sorted_f_imp

{'ost_prob_var': 23.93336947223933,
 'ost_prob_mean': 23.646622704134742,
 'ost_prob_last': 15.913645603273089,
 'ost_prob_max': 15.170113554434245,
 'tara': 12.560540066819648,
 'cnsi_volumek': 8.775708599098945}

In [221]:
list(sorted_f_imp.keys())

['ost_prob_var',
 'ost_prob_mean',
 'ost_prob_last',
 'ost_prob_max',
 'tara',
 'cnsi_volumek']

In [222]:
wag_param.columns

Index(['wagnum', 'model', 'rod_id', 'gruz', 'cnsi_gruz_capacity',
       'cnsi_volumek', 'tara', 'date_build', 'srok_sl', 'zavod_build',
       'date_iskl', 'cnsi_probeg_dr', 'cnsi_probeg_kr', 'kuzov', 'telega',
       'tormoz', 'tipvozd', 'tippogl', 'norma_km', 'ownertype'],
      dtype='object')

In [229]:
import os
import h2o
import xgboost as xgb
import pandas as pd
from h2o.automl import H2OAutoML

h2o.init()

model_path = ""
model_file = os.path.join(model_path, "StackedEnsemble_AllModels_1_AutoML_2_20231111_112553")
train = h2o.H2OFrame(X_train)
test = h2o.H2OFrame(X_test)
train[y_month_name] = train[y_month_name].asfactor()
test[y_month_name] = test[y_month_name].asfactor()

x = train.columns
y = y_month_name
x.remove(y)

# if os.path.exists(model_file):
#     print("sdfsd")
#     aml = h2o.load_model(model_file)
# else:
aml = H2OAutoML(max_models=15, seed=42)
aml.train(x=x, y=y, training_frame=train)

h2o.save_model(aml.leader, path=model_path)

lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 19.0.2+7-44, mixed mode, sharing)
  Starting server from D:\All_python_trash\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Dforgeek\AppData\Local\Temp\tmpz0afjv4g
  JVM stdout: C:\Users\Dforgeek\AppData\Local\Temp\tmpz0afjv4g\h2o_Dforgeek_started_from_python.out
  JVM stderr: C:\Users\Dforgeek\AppData\Local\Temp\tmpz0afjv4g\h2o_Dforgeek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,2 months and 20 days
H2O_cluster_name:,H2O_from_python_Dforgeek_1qkv13
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.984 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
23:02:24.141: AutoML: XGBoost is not available; skipping it.
███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_AllModels_1_AutoML_1_20231111_230224     0.972437  0.0787833  0.723226                0.181747  0.15362   0.023599
StackedEnsemble_BestOfFamily_1_AutoML_1_20231111_230224  0.972305  0.0788417  0.722483                0.184622  0.153796  0.0236533
GBM_1_AutoML_1_20231111_230224                           0.971878  0.0796774  0.721355                0.176236  0.154126  0.0237547
GBM_2_AutoML_1_20231111_230224                           0.971565  0.0808639  0.716726                0.170618  0.155238  0.02

In [230]:
y_pred = aml.predict(test).as_data_frame().squeeze()
result = f1_score(X_test[y_month_name], y_pred["predict"])
print(f"F1: {result}")


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
F1: 0.629018245004344


In [225]:
# h2o.save_model(aml.leader, path=f"saved_models/h20_{result}")
# h2o.shutdown()

In [226]:
# feature_importance = logreg.coef_
# 
# avg_feature_importance = np.mean(np.abs(feature_importance), axis=0)
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": avg_feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [227]:
# feature_importance = rf.feature_importances_
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [228]:
# import shap
# 
# explainer = shap.TreeExplainer(rf)
# shap_values = explainer.shap_values(X_train)
# shap.summary_plot(shap_values, X_train, plot_type="bar")